$\textbf{Questions :}$v A first example of MILP [20 marks]
As a student in IITB, you are involved in selecting courses for completing your course requirements.
You also want to make sure that the courses you select are interesting to you. Suppose that there
are ten courses C0, C1, C2, . . ., C9 being offered this semester. By completing each course you
get the following credits

$\begin{matrix}
Course &  Credit \\
C0 & 4 \\
C1 & 3 \\
C2 & 6 \\
C3 & 5 \\
C4 & 2 \\
C5 & 6 \\
C6 & 4 \\
C7 & 8 \\
C8 & 8 \\
C9 & 6
\end{matrix}$


You must gather at least 26 credits this semester. However your course selection depends on the
following constraints:

$\huge{\cdot}$ You must register for at least 4 courses and can register for up to a maximum of 8 courses
this semester.

$\huge{\cdot}$ Courses C2 and C7 run during the same slots.

$\huge{\cdot}$ Courses C1 and C9 run during the same slots.

$\huge{\cdot}$ If you register for course C2 you cannot register for course C8.

$\huge{\cdot}$ If you register for courses C9 you must register for course C4.

$\huge{\cdot}$ Among the courses C0, C1, C2, C3, you must register for at least one course and can only
register for at most two courses.

$\huge{\cdot}$ Among the courses C5, C9, you can only register at most one course.

$\huge{\cdot}$ Among the courses C6, C8, C9, you can only register for at most two courses.

$\textbf{1.}$ Write a mathematical model so that you can maximize your credits to be accumulated
this semester. Explain how to design the model variables and indicate which variables are
integers. (We usually replace the notation x ∈ R by x ∈ Z to denote that variable x is
integer).

$\textbf{Solution :-} $



$\textbf{ Mathematical Formulation :} \\ 
 \text{max} \ 4x_0+3x_1+6x_2+5x_3+2x_4+6x_5+4x_6+8x_7+8x_8+6x_9 \\
 4x_0+3x_1+6x_2+5x_3+2x_4+6x_5+4x_6+8x_7+8x_8+6x_9>=26 \\
 4<=\sum_{i=0}^{9}x_i<=8, \\
 x_2+x_7<=1, \\
 x_1+x_9<=1, \\
 1<=\sum_{i=0}^{3}<=2, \\
 x_5+x_9<=1, \\
 x_6+x_8+x_9<=2, \\ 
 x_9-x_4<=0, \\
 x_2+x_8<=1, \\
 x_i \in  \{0,1\} , where \ i=0,1,2,...,9$


$\text{Here,} x_i ( where \ i \in \{0,1,...,9\})  \ \text{denote we are going to taking corresponding course} \ C_i ,i \in \{0,1,...,9\} \ \text{or  not.}  $

so, $x_i$ should be either $0$ or $1$, here $0$ denote we are not goging to taking that course and $1$ means we are goging to take that course.

 $\mathbf{2.}$ Use pyomo to construct your model.

In [158]:
!pip install -q pyomo
from pyomo.environ import *
import numpy as np

In [159]:
model = ConcreteModel()

In [160]:
coef_A = np.array([[-4,-3,-6,-5,-2,-6,-4,-8,-8,-6],
                   [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
                   [1,1,1,1,1,1,1,1,1,1],
                  [1,1,1,1,0,0,0,0,0,0],
                  [-1,-1,-1,-1,0,0,0,0,0,0],
                  [0,0,1,0,0,0,0,1,0,0],
                  [0,1,0,0,0,0,0,0,0,1],
                  [0,0,0,0,0,1,0,0,0,1],
                  [0,0,0,0,0,0,1,0,1,1],
                  [0,0,0,0,-1,0,0,0,0,1],
                  [0,0,1,0,0,0,0,0,1,0]])

coef_rhs_b = np.array([-26,-4,8,2,-1,1,1,1,2,0,1])

In [161]:
n = 10
m = 11
col_indices = np.arange(n)


In [162]:
model.x = Var(col_indices,domain=Binary)

In [163]:
model.constraints = ConstraintList()

In [164]:
for i in range(m):
  model.constraints.add(sum(coef_A[i][j]*model.x[j] for j in col_indices)<=coef_rhs_b[i])

In [165]:
model.objective = Objective(expr = sum((-1)*coef_A[0][i]*model.x[i] for i in col_indices),sense = maximize)

In [166]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

1 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None :   

$\mathbf{3.}$ [R] Solver your MILP using cbc solver and report the optimal values of the variables and the
objective function value.


$\textbf{Solution : -}$ 

In [167]:
!apt-get install -y -qq coinor-cbc

In [168]:
opt = SolverFactory('cbc')
#.solve(model1).write()
result = opt.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 37.0
  Upper bound: 37.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 10
  Number of binary variables: 10
  Number of integer variables: 10
  Number of nonzeros: 10
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.02113628387451172
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [169]:
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
print('\nConstraints')
model.constraints.display()


Objective =  37.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  1.0
x[ 6 ] =  1.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  0.0

Constraints
constraints : Size=11
    Key : Lower : Body  : Upper
      1 :  None : -37.0 : -26.0
      2 :  None :  -7.0 :  -4.0
      3 :  None :   7.0 :   8.0
      4 :  None :   2.0 :   2.0
      5 :  None :  -2.0 :  -1.0
      6 :  None :   1.0 :   1.0
      7 :  None :   0.0 :   1.0
      8 :  None :   1.0 :   1.0
      9 :  None :   2.0 :   2.0
     10 :  None :  -1.0 :   0.0
     11 :  None :   1.0 :   1.0


$\mathbf{4} $. [R] Let us now compare it to a linear program. Suppose we remove the restrictions that
the variables in the above problem are integers. Solve this modified problem and report the
optimal values of the variables and the objective function value.

$\textbf{Solution :- } $


In [170]:
model1 = ConcreteModel()

In [171]:
model1.y = Var(col_indices,domain=NonNegativeReals)

In [172]:
model1.constraints = ConstraintList()

In [173]:
for i in range(m):
  model1.constraints.add(sum(coef_A[i][j]*model1.y[j] for j in col_indices)<=coef_rhs_b[i])

In [174]:
upper_bound = np.array([1,1,1,1,1,1,1,1,1,1])

for i in col_indices:
  model1.y[i].setub(upper_bound[i])

In [175]:
model1.objective = Objective(expr = sum((-1)*coef_A[0][i]*model1.y[i] for i in col_indices),sense = maximize)

In [176]:
model1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    y_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

1 Var Declarations
    y : Size=10, Index=y_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : NonNegativeReals
          1 :     0 :  None :     1 : False :  True : NonNegativeReals
          2 :     0 :  None :     1 : False :  True : NonNegativeReals
          3 :     0 :  None :     1 : False :  True : NonNegativeReals
          4 :     0 :  None :     1 : False :  True : NonNegativeReals
          5 :     0 :  None :     1 : False :  True : NonNegativeReals
          6 :     0 :  None :     1 : False :  True : NonNegativeReals
          7 :     0 :  

In [177]:
result1 = opt.solve(model1)
print(result1)

print('Solver status:', result1.solver.status)
print('Solver termination condition:',result1.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 37.0
  Upper bound: 37.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 11
  Number of nonzeros: 10
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 4
  Error rc: 0
  Time: 0.024727582931518555
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [178]:
print('\nObjective = ', model1.objective())

print('\nDecision Variables')
for i in col_indices:
  print('y[',i,'] = ', model1.y[i].value)
print('\nConstraints')
model1.constraints.display()


Objective =  37.0

Decision Variables
y[ 0 ] =  1.0
y[ 1 ] =  0.0
y[ 2 ] =  0.0
y[ 3 ] =  1.0
y[ 4 ] =  1.0
y[ 5 ] =  1.0
y[ 6 ] =  1.0
y[ 7 ] =  1.0
y[ 8 ] =  1.0
y[ 9 ] =  0.0

Constraints
constraints : Size=11
    Key : Lower : Body  : Upper
      1 :  None : -37.0 : -26.0
      2 :  None :  -7.0 :  -4.0
      3 :  None :   7.0 :   8.0
      4 :  None :   2.0 :   2.0
      5 :  None :  -2.0 :  -1.0
      6 :  None :   1.0 :   1.0
      7 :  None :   0.0 :   1.0
      8 :  None :   1.0 :   1.0
      9 :  None :   2.0 :   2.0
     10 :  None :  -1.0 :   0.0
     11 :  None :   1.0 :   1.0


$\mathbf{5.}$ Can the solution of the MILP be obtained by merely rounding the solution of the LP?
Why or why not?


$\textbf{Solution :-}$

it is very rare to get solution of the MILP be obtained by rounding the solution of the Lp ,some time if you going to rounding the solution of Lp you may find that the new generating solution is not satisfying your constraints. even some time it may be satisfy but you may find this is not optimal solution. so rounding the solution of lp to get solution of MILP is not a way to finding solution of MILP

$\mathbf{6.}$ Suppose a new course C10 with 12 credits is introduced, but the course timing for C10
overlaps with the course timings of C5, C6, how will you change your model to include this
new constraint?

$\textbf{Solution :-}$

because adding new course i need 1 extra variable 
let's take x10 which denote we are goging to take course c10 or not 
and we need to add extra constraint to our above lpp which is 


$x_{10} +x_5 <=1 ,\\
x_{10}+x_6<=1$

and also need to add $12x_{10} $ in our objective function.

 $\mathbf{7.}$ Solve the modified MILP and report the optimal values of the variables and objective
function value.


$\textbf{Solution :-}$

In [179]:
model.objective.deactivate()

In [180]:
model.x10=Var(domain=Binary)

In [181]:
model.new_objective = Objective(expr = sum((-1)*coef_A[0][i]*model.x[i] for i in col_indices)+12*model.x10,sense=maximize)

In [182]:
model.constraints.add(model.x10+model.x[5]<=1)
model.constraints.add(model.x10+model.x[6]<=1)

In [183]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   13 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

2 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False :  True : Binary
          1 :     0 :   0.0 :     1 : False :  True : Binary
          2 :     0 :   0.0 :     1 : False :  True : Binary
          3 :     0 :   1.0 :     1 : False :  True : Binary
          4 :     0 :   1.0 :     1 : False :  True : Binary
          5 :     0 :   1.0 :     1 : False :  True : Binary
          6 :     0 :   1.0 :     1 : False :  True : Binary
          7 :     0 :   1.0 :     1 : False :  True : Binary
          8 :     0 :   

In [185]:
result2 = opt.solve(model)
print(result2)

print('Solver status:', result2.solver.status)
print('Solver termination condition:',result2.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 45.0
  Upper bound: 45.0
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 11
  Number of binary variables: 11
  Number of integer variables: 11
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.020402193069458008
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [187]:
print('\nObjective = ', model.new_objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
print('x[ 10 ] = ',model.x10.value)
print('\nConstraints')
model.constraints.display()


Objective =  45.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  0.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  0.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  1.0
x[ 10 ] =  1.0

Constraints
constraints : Size=13
    Key : Lower : Body  : Upper
      1 :  None : -33.0 : -26.0
      2 :  None :  -6.0 :  -4.0
      3 :  None :   6.0 :   8.0
      4 :  None :   2.0 :   2.0
      5 :  None :  -2.0 :  -1.0
      6 :  None :   1.0 :   1.0
      7 :  None :   1.0 :   1.0
      8 :  None :   1.0 :   1.0
      9 :  None :   2.0 :   2.0
     10 :  None :   0.0 :   0.0
     11 :  None :   1.0 :   1.0
     12 :  None :   1.0 :   1.0
     13 :  None :   1.0 :   1.0


$\mathbf{8.}$ [R] Solve the modified MILP by removing the integer constraints on the variables and check
if the solution of the modified MILP can be obtained by merely rounding the solution of the
corresponding modified LP? Explain.


In [188]:
model2 = ConcreteModel()

In [189]:
model2.y = Var(col_indices,domain=NonNegativeReals)
model2.y10=Var(domain=NonNegativeReals)

In [190]:
model2.constraints = ConstraintList()

In [192]:
for i in range(m):
  model2.constraints.add(sum(coef_A[i][j]*model2.y[j] for j in col_indices)<=coef_rhs_b[i])

model2.constraints.add(model2.y10+model2.y[5]<=1)
model2.constraints.add(model2.y10+model2.y[6]<=1)

In [194]:
upper_bound = np.array([1,1,1,1,1,1,1,1,1,1])

for i in col_indices:
  model2.y[i].setub(upper_bound[i])
model2.y10.setub(1)

In [195]:
model2.objective = Objective(expr = 12*model2.y10+sum((-1)*coef_A[0][i]*model2.y[i] for i in col_indices),sense = maximize)

In [196]:
result1 = opt.solve(model2)
print(result1)

print('Solver status:', result1.solver.status)
print('Solver termination condition:',result1.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 45.0
  Upper bound: 45.0
  Number of objectives: 1
  Number of constraints: 25
  Number of variables: 12
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 6
  Error rc: 0
  Time: 0.02195906639099121
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [197]:
print('\nObjective = ', model2.objective())

print('\nDecision Variables')
for i in col_indices:
  print('y[',i,'] = ', model2.y[i].value)
print('\nConstraints')
model1.constraints.display()


Objective =  45.0

Decision Variables
y[ 0 ] =  1.0
y[ 1 ] =  0.0
y[ 2 ] =  0.0
y[ 3 ] =  1.0
y[ 4 ] =  1.0
y[ 5 ] =  0.0
y[ 6 ] =  0.0
y[ 7 ] =  1.0
y[ 8 ] =  1.0
y[ 9 ] =  1.0

Constraints
constraints : Size=11
    Key : Lower : Body  : Upper
      1 :  None : -37.0 : -26.0
      2 :  None :  -7.0 :  -4.0
      3 :  None :   7.0 :   8.0
      4 :  None :   2.0 :   2.0
      5 :  None :  -2.0 :  -1.0
      6 :  None :   1.0 :   1.0
      7 :  None :   0.0 :   1.0
      8 :  None :   1.0 :   1.0
      9 :  None :   2.0 :   2.0
     10 :  None :  -1.0 :   0.0
     11 :  None :   1.0 :   1.0


here i get same solution as when i solve for Ipp and all the variable are already integer so no need to rounding . so we can say here that without using binary bound we got same answer. but this is rare most of the time we not get same answer even after rounding the lpp optimal variable.